# Plot Concentration Bound
---

## Import Packages

In [1]:
import math
import numpy as np

from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme(style="darkgrid")

# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm
# from matplotlib.ticker import LinearLocator, FormatStrFormatter
# import matplotlib.pyplot as plt

## Global Parameters

In [2]:
# N = 10**4
k = 100
print(N)
print(k)

NameError: name 'N' is not defined

# The Risk Bound

In [3]:
def get_risk_bound__scalar(N,k,F):
    #-----------------------------------------#
    ## Case-By-Case Definition of Quantities ## 
    #-----------------------------------------#
    if F == 1:
        # rate
        rate = 1/np.sqrt(N)
        # C_F
        C_F = 1/(np.sqrt(8)-2)
        # dist
        dist = 12*k
    if F == 2:
        # rate
        rate = (32+math.log2(N))/np.sqrt(N)
        # C_F
        C_F = np.sqrt(2)/4
        # dist
        dist = 12*k
    if F > 2:
        # rate
        rate = 1/(N**(1/F))
        # C_F
        C_F__1 = 2
        C_F__2 = ((F/2)-1)/(2*(1-2**(1-(F/2))))
        C_F__2 = C_F__2**(2/F)
        C_F__3 = (1+1/(2*((F/2)-1)))
        C_F__4 = np.sqrt(F)
        C_F = C_F__1*C_F__2*C_F__3*C_F__4

        if F <= 8*np.round(math.log(k),0):
            dist = 15*np.sqrt(math.log(k))*(k**(2/F))
            dist = dist/(np.sqrt(F))
        if F > 8*np.round(math.log(k),0):
            dist__1 = 48*math.floor(math.log2(k)+1)
            dist__2_1 = np.sqrt(F)+ 2*np.sqrt(2)*np.sqrt(np.log(k))
            dist__2_2 = np.sqrt(F)- 2*np.sqrt(2)*np.sqrt(np.log(k))
            dist__2 = np.sqrt((dist__2_1/dist__2_2))
            dist = dist__1*dist__2

    #-----------------#
    ## Compute Bound ## 
    #-----------------#
    bound = dist*C_F*rate

    return bound

Vectorize:

In [4]:
get_risk_bound = np.vectorize(get_risk_bound__scalar)

## Generate Plot

### Fixed k 

In [59]:
# Set External Values of k and F
k_global = 10**3

N_min = 10**1
N_max = 10**3
N_tests = 10**3


# Generate Sequence of Number of Samples
N_sequence = np.linspace(N_min,N_max,N_tests)

In [61]:
plt.figure(figsize=(6,6))

plt.title('k='+str(k_global)+', Varying F and N')
plt.xlabel('No. Samples (N)')
plt.ylabel('Bound on $\mathbb{E}[\mathcal{W}_1(\mathbb{P},\mathbb{P}^N)]$')

for i in range(1,40,5):
    # Set Value of F
    F_global = int(np.round(i,0))
    
    # Function of N only
    def get_risk_bound__NVar_scalar(N):
        return get_risk_bound__scalar(k=k_global,F=F_global,N=N)
    
    get_risk_bound__NVar = np.vectorize(get_risk_bound__NVar_scalar)
    
    # Compute Risk Bound
    Risk_Bounds = get_risk_bound__NVar_scalar(N_sequence)
    
    # Plot
    plt.plot(N_sequence, Risk_Bounds,label = 'F='+str(F_global).format(i=i))
    
plt.legend(loc='best')

plt.savefig('NF_kis'+str(k_global)+'NMin'+str(N_min)+'NMax'+str(N_max)+'.pdf')
plt.show()

### Fixed N

In [147]:
# Set External Values of k and F
N_global = 10**5

F_min = 4
F_max = 10**5
F_tests = 50


# Generate Sequence of Number of Samples
F_sequence = np.linspace(F_min,F_max,F_tests)

In [35]:
plt.figure(figsize=(6,6))

plt.title('N='+str(N_global)+', Varying k and F')
plt.xlabel('Embedding Dimension (F)')
plt.ylabel('Bound on $\mathbb{E}[\mathcal{W}_1(\mathbb{P},\mathbb{P}^N)]$')

for i in range(4,1000,200):
    # Set Value of F
    k_global = int(np.round(i,0))
    
    # Function of N only
    def get_risk_bound__NVar_scalar(F):
        return get_risk_bound__scalar(k=k_global,F=F,N=N_global)
    
    get_risk_bound__FVar = np.vectorize(get_risk_bound__NVar_scalar)
    
    # Compute Risk Bound
    Risk_Bounds = get_risk_bound__FVar(F_sequence)
    
    # Plot
    plt.plot(F_sequence, Risk_Bounds,label = 'k='+str(k_global).format(i=i))
    
plt.legend(loc='best')

plt.savefig('kF_Nis'+str(N_global)+'.pdf')
plt.show()

NameError: name 'N_global' is not defined

<Figure size 432x432 with 0 Axes>

### Get Mesh

In [31]:
f = arange(1,100,1)
n = arange(100,10000,1)
K,F = meshgrid(f,n) # grid of point
Z = get_risk_bound(K, F) # evaluation of the function on the grid

### Get Plot

In [32]:
fig,ax=plt.subplots(1,1)

cp = ax.contourf(K, F, Z)
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('$\mathbb{E}[\mathcal{W}_1(\mathbb{P},\mathbb{P}^N)]$-Bound vs. k and F for N='+str(N))
ax.set_xlabel('F')
ax.set_ylabel('N')

# plt.savefig('kF_Nis'+str(N)+'.pdf')
plt.savefig('NF_kis'+str(k)+'.pdf')
plt.show()

---
# Fin #
---